# Cross validation of the ALS method using Spark
We refer to this tutorial: http://spark.apache.org/docs/latest/ml-tuning.html#example-model-selection-via-cross-validation

The tutorial was not clear: we used SKLearn to split the dataset

# Import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


In [2]:
from helpers_cross_validation import KFoldIndexes

In [3]:
import random


In [4]:
from pyspark.sql.functions import col

cross validation imports

In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [16]:
import itertools

In [17]:
?itertools.combinations

In [15]:
class CrossValidation:
    ''' Class to run the cross validation with different models
    
    Usage:
    cross_val=CrossValidation(data,k,use_spark)
    cross_val.fit(model)
    
    Variables:
    self.tests_list
    '''
    
    
    def __init__(data,k,use_spark):
        
        # Initialize the parameters
        self.k=k
        self.use_spark=use_spark
        
        # Initialize the k_fold_indexes
        k_fold_indexes=KFoldIndexes(k_folds,data.shape[0])
        
        if use_spark:
            self.tests_list=get_sql_from_pd(get_tests_database(data,k_fold_indexes))
        else:
            self.tests_list=get_tests_database(data,k_fold_indexes)
    
    def fit(model,**arg):
        for comb in itertools.combinations(range(self.k),self.k-1):
            trains=[self.tests_list for x in comb]
            train=sc.union(trains)
            
            test_index=[x for x in range(self.k) if x not in comb][0]
            test=self.tests_list[test_index]
            
            model.fit(train,**arg)
            predictions=model.predict(test)
            
            
    
    def get_tests_database(data,k_fold_indexes):
        tests=[]
        for i in k_fold_indexes.indexes:
            tests.append(data.loc[i[1]])
        return tests
    
    def get_sql_from_pd(df_list):
        sql_list=[]
        for i in df_list:
            sql_list.append(sqlContext.createDataFrame(i).rdd)
        return sql_list
    

# Dataframe creation

In [6]:
train = pd.read_csv('../data/data_train.csv')


In [7]:
train['UserID'] = train['Id'].apply(lambda x: int(x.split('_')[0][1:]))
train['MovieID'] = train['Id'].apply(lambda x: int(x.split('_')[1][1:]))
train['Rating'] = train['Prediction']
train = train.drop(['Id', 'Prediction'], axis=1)

In [8]:
train.shape[0]

1176952

In [10]:
k_folds=4
k_fold_indexes=KFoldIndexes(k_folds,train.shape[0])

In [ ]:
train.loc[[1,2,3]]

In [ ]:
def get_tests_database(df,k_fold_indexes):
    tests=[]
    for i in k_fold_indexes.indexes:
        tests.append(df.loc[i[1]])
    return tests

In [ ]:
def get_sql_from_pd(df_list):
    sql_list=[]
    for i in df_list:
        print(1)
        sql_list.append(sqlContext.createDataFrame(i))
    return sql_list

In [ ]:
tests=get_tests_database(train,k_fold_indexes)

In [ ]:
tests_sql=get_sql_from_pd(tests)

In [ ]:
tr=tests_sql[0].rdd.union(tests_sql[1].rdd).union(tests_sql[2].rdd)

In [ ]:
ts=tests_sql[3].rdd

In [ ]:
model = ALS.train(tr, 2, 10, 0.1)

In [ ]:
validation_for_predict_RDD = ts.map(lambda x: (x[0], x[1]))

In [ ]:
predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
rates_and_preds = ts.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)

In [ ]:
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())

In [ ]:
error

In [ ]:
tests_sql

In [ ]:
train_sql = sqlContext.createDataFrame(train)
train_rdd = train_sql.rdd
train_rdd.take(1)

In [ ]:
type(train_sql.map(lambda x: (x[0],x[1],float(x[2]))))